<a href="https://colab.research.google.com/github/melsubair/Subair/blob/master/covid_19_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import scipy
import warnings
import numpy as np
from tqdm import tqdm
from random import shuffle
import matplotlib.pyplot as plt

from keras.optimizers import Adam, RMSprop
from keras.models import Sequential, Model
from keras.layers import  Conv2D, MaxPooling2D, Activation, Flatten,Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

Using TensorFlow backend.


In [2]:
# Filter Warnings
warnings.filterwarnings("ignore")

In [3]:
# File Paths
input_path = "../covid-19-xray/"

In [5]:
# File Contents
for _set in ['train', 'test']:
    normal = len(os.listdir(input_path + _set + '/normal'))
    pneumonia = len(os.listdir(input_path + _set + '/pneumonia'))
    print('The {} folder contains {} Normal and {} Pneumonia images.'.format(_set, normal, pneumonia))

The train folder contains 1336 Normal and 1615 Pneumonia images.
The test folder contains 234 Normal and 390 Pneumonia images.


In [6]:
# Preprocesing Data Function
def preprocess_data(input_path, img_dims, batch_size):
    
    # Data Augmentation for Train & Test Images
    train_datagen = ImageDataGenerator(
        rescale = 1./255,
        zoom_range = 0.2,
        shear_range = 0.2,      
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(
        rescale = 1./255)
    
    train_images = train_datagen.flow_from_directory(
        directory = input_path + 'train', 
        target_size = (img_dims, img_dims), 
        batch_size = batch_size, 
        class_mode = 'binary')

    test_images = test_datagen.flow_from_directory(
        directory = input_path + 'test', 
        target_size = (img_dims, img_dims), 
        batch_size = batch_size, 
        class_mode = 'binary')

    # I'm created these lists for make prediction on test image and showing confusion matrix.
    train_labels = []
    test_labels = []

    for file_name in ['/normal/', '/pneumonia/']:
        for img in (os.listdir(input_path + 'test' + file_name)):
            img = cv2.imread(input_path + 'test' + file_name + img, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if file_name == '/normal/':
                label = 0
            elif file_name == '/pneumonia/':
                label = 1
            train_labels.append(img)
            test_labels.append(label)
        
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    
    return train_images, train_labels, test_images, test_labels

In [7]:
img_dims = 150
epochs = 30
batch_size = 2

# Set Images&Labels for Train,Test
train_images, train_labels, test_images, test_labels = preprocess_data(input_path, img_dims, batch_size)

Found 2951 images belonging to 3 classes.
Found 624 images belonging to 3 classes.
